In [1]:
# imports
import os
import torch 
import torchvision
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [2]:
# constants
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
BATCH_SIZE = 16
NOISE_FACTOR = 0.5

In [3]:
# transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

data_preprocess_transform = transforms.Compose(
    [torchvision.transforms.Resize((256,256)), torchvision.transforms.ToTensor()
     ])

In [4]:
trainset = torchvision.datasets.VOCSegmentation(
        root='/Users/williambjorndahl/Downloads',
        year= '2012',
        image_set = 'train',
        download=False,
        transform=data_preprocess_transform,
        target_transform=data_preprocess_transform)

testset = torchvision.datasets.VOCSegmentation(
        root='/Users/williambjorndahl/Downloads',
        year= '2012',
        image_set='trainval',
        download=False,
        transform=data_preprocess_transform,
        target_transform=data_preprocess_transform)

trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2)

testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2)

In [5]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
def make_dir():
    image_dir = 'Saved_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
def save_decoded_image(img, name):
    img = img[0]
    save_image(img, name)

In [6]:
# the autoencoder network
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder layers
        self.enc1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.enc2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.enc3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.enc4 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # decoder layers
        self.dec1 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2)
        self.dec2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2)
        self.dec3 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.dec4 = nn.ConvTranspose2d(16, 3, kernel_size=2, stride=2)
        self.out = nn.Conv2d(3, 3, kernel_size=3, padding=1)
        self.upsample = torch.nn.Upsample((256, 256), mode='bilinear')
    def forward(self, x):
        # encode
        x = F.relu(self.enc1(x))
        x = self.pool(x)
        x = F.relu(self.enc2(x))
        x = self.pool(x)
        x = F.relu(self.enc3(x))
        x = self.pool(x)
        x = F.relu(self.enc4(x))
        x = self.pool(x)  # the latent space representation

        # decode
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.sigmoid(self.out(x))

        x = torch.nn.functional.interpolate(x, size=(256, 256), mode='bilinear')
        return x
net = Autoencoder()
print(net)

Autoencoder(
  (enc1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (enc2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (enc3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (enc4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dec1): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
  (dec2): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
  (dec3): ConvTranspose2d(32, 16, kernel_size=(2, 2), stride=(2, 2))
  (dec4): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
  (out): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (upsample): Upsample(size=(256, 256), mode=bilinear)
)


In [7]:
# the loss function
criterion = nn.MSELoss()
# the optimizer
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [8]:
# the training function
def train(net, trainloader, NUM_EPOCHS):
    train_loss = []
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        for data in enumerate(trainloader):
            idx, (img, _) = data # we do not need the image labels
            print('img',img.shape)
            # add noise to the image data
            img_noisy = img + NOISE_FACTOR * torch.randn(img.shape)
            # clip to make the values fall between 0 and 1
            img_noisy = np.clip(img_noisy, 0., 1.)
            img_noisy = img_noisy.to(device)
            optimizer.zero_grad()
            outputs = net(img_noisy)
            print('outputs',outputs.shape)
            print('noisy',img_noisy.shape)
            loss = criterion(outputs, img_noisy)
            # backpropagation
            loss.backward()
            # update the parameters
            optimizer.step()
            running_loss += loss.item()
        
        loss = running_loss / len(trainloader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(
            epoch+1, NUM_EPOCHS, loss))
        save_decoded_image(img_noisy.cpu().data, name='./Saved_Images/noisy{}.png'.format(epoch))
        save_decoded_image(outputs.cpu().data, name='./Saved_Images/denoised{}.png'.format(epoch))
    return train_loss

In [9]:
def test_image_reconstruction(net, testloader):
     for batch in testloader:
        img, _ = batch
        img_noisy = img + NOISE_FACTOR * torch.randn(img.shape)
        img_noisy = np.clip(img_noisy, 0., 1.)
        img_noisy = img_noisy.to(device)
        outputs = net(img_noisy)
        save_image(outputs[0], 'eval_denoised.png')
        save_image(img_noisy, 'noisy_test_input.png')
        save_image(outputs, 'denoised_test_reconstruction.png')
        break

In [10]:
device = get_device()
print(device)
net.to(device)
make_dir()
train_loss = train(net, trainloader, NUM_EPOCHS)
plt.figure()
plt.plot(train_loss)
plt.title('Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('./Saved_Images/conv_ae_fahsionmnist_loss.png')
test_image_reconstruction(net, testloader)

cpu
img torch.Size([16, 3, 256, 256])


/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3060: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 2

outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 2

img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 

...

img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 

img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 

img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([8, 3, 256, 256])
outputs torch.Size([8, 3, 256, 256])
noisy torch.Size([8, 3, 256, 256])
Epoch 5 of 10, Train Loss: 0.123
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256]

img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 256])
outputs torch.Size([16, 3, 256, 256])
noisy torch.Size([16, 3, 256, 256])
img torch.Size([16, 3, 256, 

KeyboardInterrupt: 

In [ ]:
outputs.size(0)